In [1]:
using ForwardDiff, PreallocationTools, Interpolations, LinearAlgebra
using BenchmarkTools

nₓ = 4
nₜ = 1024
Δt = 0.01

x_bound = 2 * π * nₓ / (nₓ + 1)
x_range = range(0, x_bound, nₓ)

# preallocations
coords_arr = zeros(3 * nₓ^3)
coords_new_arr = zeros(3 * nₓ^3)
u_arr = zeros(3 * nₓ^3)
u_new_arr = zeros(3 * nₓ^3)
pressure = zeros(nₓ, nₓ, nₓ)

# These cache memory for the automatic differentiation
# and store references to the original allocation
coords_arr_d = DiffCache(coords_arr)
coords_new_arr_d = DiffCache(coords_new_arr)
u_arr_d = DiffCache(u_arr)
u_new_arr_d = DiffCache(u_new_arr)
pressure_d = DiffCache(pressure)

# reshapes 1D array into vector of 3D arrays, with no allocation
function get_views(u)
    @views begin
        u_x = reshape(u[1:nₓ^3], (nₓ, nₓ, nₓ))
        u_y = reshape(u[nₓ^3+1:2*nₓ^3], (nₓ, nₓ, nₓ))
        u_z = reshape(u[2*nₓ^3+1:end], (nₓ, nₓ, nₓ))
    end
    return [u_x, u_y, u_z]
end

coords = get_views(coords_arr)
u = get_views(u_arr)

# wouldn't normally do it this way, but we need these later for advection
coords[1] .= [x for x ∈ x_range, y ∈ x_range, z ∈ x_range]
coords[2] .= [y for x ∈ x_range, y ∈ x_range, z ∈ x_range]
coords[3] .= [z for x ∈ x_range, y ∈ x_range, z ∈ x_range]

# figure out how to cache these
# interpolators = map(u) do x
#     interpolate((x_range, x_range, x_range), x, Gridded(Linear()))
# end;
# interpolators_d = DiffCache(interpolators)

4×4×4 reshape(view(::Vector{Float64}, 129:192), 4, 4, 4) with eltype Float64:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 1.67552  1.67552  1.67552  1.67552
 1.67552  1.67552  1.67552  1.67552
 1.67552  1.67552  1.67552  1.67552
 1.67552  1.67552  1.67552  1.67552

[:, :, 3] =
 3.35103  3.35103  3.35103  3.35103
 3.35103  3.35103  3.35103  3.35103
 3.35103  3.35103  3.35103  3.35103
 3.35103  3.35103  3.35103  3.35103

[:, :, 4] =
 5.02655  5.02655  5.02655  5.02655
 5.02655  5.02655  5.02655  5.02655
 5.02655  5.02655  5.02655  5.02655
 5.02655  5.02655  5.02655  5.02655

In [2]:
function TaylorGreen!(u, coords)
    u_x, u_y, u_z = u
    x, y, z = coords
    @. u_x =  sin(x) * cos(y) * cos(z)
    @. u_y = -cos(x) * sin(y) * cos(z)
    @. u_z = 0.
    return
end
TaylorGreen!(u, coords)

In [11]:
function Base.mod1(I::CartesianIndex, J::CartesianIndex)
    tup = broadcast(mod1, Tuple(I), Tuple(J))
    return CartesianIndex(tup)
end

function δ(I::CartesianIndex{N}, dim) where N
    return CartesianIndex(ntuple(i -> i == dim ? 1 : 0, N))
end

function ∂(A, dim)
    out = similar(A)
    R = CartesianIndices(A)
    bound = last(R)
    for I ∈ R
        I⁻ = mod1(I - δ(I, dim), bound)
        out[I] = A[I] - A[I⁻]
    end
    return out
end

∂ (generic function with 1 method)

In [12]:
I = CartesianIndex(1, 1, 1)
J = CartesianIndex(4, 4, 4)
δ(I, 1)

CartesianIndex(1, 0, 0)

In [13]:
mod1(I - δ(I, 1), J)

MethodError: MethodError: no method matching mod1(::Int64, ::Int64)
You may have intended to import Base.mod1

In [3]:
function advect_semi_lagrangian!(u_arr, coords_cache, coords_new_cache, Δt)
    # get cached arrays (either real or dual, depending on typeof(u))
    u = get_views(u_arr)
    coords = get_tmp(coords_cache, u_arr)
    coords_new = get_tmp(coords_new_cache, u_arr)

    # compute self advection of u
    # initialize interpolators
    interpolators = map(u) do x
        interpolate((x_range, x_range, x_range), x, Gridded(Linear()))
    end
    
    # (1) trace each grid point backwards along u
    # mod1 takes care of PBC
    @. coords_new = mod1(coords - Δt * u_arr, x_bound)

    # (2) interpolate the velocity field at the new grid points
    for i in 1:3
        u[i] .= interpolators[i].(get_views(coords_new)...)
    end
    return u_arr
end



function project_incompressible(u_arr)

end

project_incompressible (generic function with 1 method)

In [4]:
u_new_arr .= u_arr
advect_semi_lagrangian!(u_new_arr, coords_arr_d, coords_new_arr_d, Δt);

In [12]:
ForwardDiff.derivative(0.) do r
    u_new = get_tmp(u_new_arr_d, r)
    @. u_new += r * u_arr
    advect_semi_lagrangian!(u_new, coords_arr_d, coords_new_arr_d, Δt);
    return u_new
end

192-element Vector{Float64}:
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
 -0.36327126400268
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [47]:
using AlgebraicMultigrid, IterativeSolvers
A = poisson(20)
A_cache = DiffCache(A)

ForwardDiff.jacobian(ones(20)) do x
    A_tmp = poisson(eltype(x), 20)
    # A_tmp = get_tmp(A_cache, x)
    # A_tmp .= A
    # ml = ruge_stuben(A_tmp)
    # AlgebraicMultigrid._solve(ml, A_tmp * x)
    # A_tmp \ (A_tmp * x)
    cg(A_tmp, A_tmp*x) # This actually works!
end

20×20 Matrix{Float64}:
  1.0          -3.86843e-16   4.97866e-16  …   8.95117e-16   1.5439e-16
 -4.44089e-16   1.0          -1.49186e-16      2.53964e-15  -9.64506e-16
 -9.54098e-16  -1.94289e-15   1.0             -1.49186e-16   2.08167e-16
 -5.82867e-16  -3.29597e-15   2.22045e-16      2.05391e-15  -9.29812e-16
 -1.81799e-15  -1.6237e-15   -2.04003e-15      2.86576e-15  -9.64506e-16
 -1.40166e-15  -4.06619e-15   7.42462e-16  …   1.65146e-15  -8.04912e-16
 -2.55351e-15  -2.56739e-15  -2.81719e-15      3.20577e-15  -1.83187e-15
 -1.16573e-15  -5.59275e-15   1.0              8.46545e-16  -9.71445e-16
 -2.9976e-15    1.0          -1.0             -1.0          -2.16493e-15
  1.0          -1.0           1.0              1.0          -1.0
 -1.0           1.0          -1.0          …  -1.0           1.0
 -2.88658e-15  -1.0           1.0              1.0          -2.69229e-15
 -1.13798e-15  -2.34535e-15  -1.0             -1.73472e-15  -1.65146e-15
 -2.8727e-15   -1.16573e-15  -1.56819e-15    

In [36]:
A_tmp = get_tmp(A_cache, b)
A_tmp

20×20 SparseArrays.SparseMatrixCSC{Float64, Int64} with 58 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦

In [23]:
A = poisson((64, 64, 64))
ml = ruge_stuben(A)

Multilevel Solver
-----------------
Operator Complexity: 2.85
Grid Complexity: 1.599
No. of Levels: 8
Coarse Solver: Pinv
Level     Unknowns     NonZeros
-----     --------     --------
    1       262144      1810432 [35.08%]
    2       131072      2417024 [46.84%]
    3        21846       707192 [13.70%]
    4         3452       180336 [ 3.49%]
    5          552        41176 [ 0.80%]
    6           85         3639 [ 0.07%]
    7           22          410 [ 0.01%]
    8            4           16 [ 0.00%]


In [25]:
Ψ = rand(size(A)[2])
@btime AlgebraicMultigrid._solve(ml, A*Ψ[:, 1])

  451.326 ms (7 allocations: 6.00 MiB)


262144-element Vector{Float64}:
 0.9579370369836959
 0.9220864723841471
 0.30769954747797706
 0.5657528159560252
 0.31869202403633273
 0.539504350895625
 0.8880858300730617
 0.4214488315304273
 0.5920146117601618
 0.9602698112006594
 ⋮
 0.24411683865740885
 0.20114826591216542
 0.8021828787599561
 0.6690684420946841
 0.7978287476774337
 0.1563696728009579
 0.5638502001080586
 0.36417366397539835
 0.1414664369750985

In [26]:
@btime A \ Ψ

  17.886 s (66 allocations: 2.33 GiB)


262144-element Vector{Float64}:
 0.4434492842503988
 0.6115094100818169
 0.6413961058450095
 0.7414367776470157
 0.7505245741236859
 0.8618739986773928
 0.9468796901642866
 0.9179074449583097
 0.984684582842769
 1.0577693704899116
 ⋮
 0.8432674179392343
 0.8582314608252583
 0.9448904247144354
 0.9039341333244569
 0.8740572405777549
 0.7111551877028455
 0.6851050130238521
 0.5136171628767003
 0.2708594867616451